In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
from sklearn import linear_model
%matplotlib inline

In [5]:
crime = pd.read_csv('NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013 - 13tbl8ny.csv',encoding='utf-8')
crime.columns = crime.iloc[3]
crime.droprow = crime.drop(crime.index[0:4], inplace=True)
crime.dropna(subset = ['Population'], inplace=True)
crime['Population'] = crime['Population'].str.replace(',','').astype(float)
crime['Population'].astype(float)
crime['Robbery'] = crime['Robbery'].str.replace(',','').astype(float)

crime = crime.replace(',','',regex=True)#crime['Property\ncrime'] = crime['Property\ncrime'].str.replace(',','')
#crime['Burglary'] = crime['Burglary'].str.replace(',','')
#crime['Robbery'] = crime['Robbery'].str.replace(',','')
crime.dtypes


3
City                                       object
Population                                float64
Violent\ncrime                             object
Murder and\nnonnegligent\nmanslaughter     object
Rape\n(revised\ndefinition)1              float64
Rape\n(legacy\ndefinition)2                object
Robbery                                   float64
Aggravated\nassault                        object
Property\ncrime                            object
Burglary                                   object
Larceny-\ntheft                            object
Motor\nvehicle\ntheft                      object
Arson3                                     object
dtype: object

In [6]:
def mukduk(x):
    if int(x) > 0:
        return int(1)
    if int(x) == 0:
        return int(0)
def robbery(x):
    if int(x) > 0:
        return int(1)
    if int(x) == 0:
        return int(0)
#print(crime.columns.to_list())
crime = crime.reset_index(drop=True)

crime['mukduk'] = crime['Murder and\nnonnegligent\nmanslaughter'].apply(mukduk)

crime['Rob'] = crime['Robbery'].apply(robbery)
    
crime['propertycrime'] = crime.Population + crime.Population*crime.Population + crime.mukduk + crime.Rob

crime.head()

3,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3,mukduk,Rob,propertycrime
0,Adams Village,1861.0,0,0,NaN,0,0.0,0,12,2,10,0,0,0,0,3.465182e+06
1,Addison Town and Village,2577.0,3,0,NaN,0,0.0,3,24,3,20,1,0,0,0,6.643506e+06
2,Akron Village,2846.0,3,0,NaN,0,0.0,3,16,1,15,0,0,0,0,8.102562e+06
3,Albany,97956.0,791,8,NaN,30,227.0,526,4090,705,3243,142,NaN,1,1,9.595476e+09
4,Albion Village,6388.0,23,0,NaN,3,4.0,16,223,53,165,5,NaN,0,1,4.081293e+07


In [14]:
from sklearn.model_selection import train_test_split
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = crime['Property\ncrime'].values.reshape(-1, 1)
X = crime[['Violent\ncrime','Robbery','Burglary']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)
regr.fit(X_train, Y_train)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_test, Y_test))
# ompute adjusted-r-squared
print(1 - (1-regr.score(X_test, Y_test))*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1))


Coefficients: 
 [[ 5.18806275 -8.91740456  2.46848594]]

Intercept: 
 [ 113.82832006]

R-squared:
0.930780577201
0.928724554742


In [7]:
df = pd.read_excel('table_8_offenses_known_to_law_enforcement_california_by_city_2013.xls')
df.columns = df.iloc[3]
df.droprow = df.drop(df.index[0:4], inplace=True)
df.dropna(subset = ['Population'], inplace=True)
df

3,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson
4,Adelanto,31165,198,2,NaN,15,52,129,886,381,372,133,17
5,Agoura Hills,20762,19,0,NaN,2,10,7,306,109,185,12,7
6,Alameda,76206,158,0,NaN,10,85,63,1902,287,1285,330,17
7,Albany,19104,29,0,NaN,1,24,4,557,94,388,75,7
8,Alhambra,84710,163,1,NaN,9,81,72,1774,344,1196,234,7
9,Aliso Viejo,50005,25,0,NaN,2,4,19,315,71,224,20,3
10,Alturas,2681,28,1,NaN,4,2,21,71,23,46,2,0
11,American Canyon,20068,54,0,NaN,4,31,19,510,91,387,32,2
12,Anaheim,345320,1130,11,NaN,82,437,600,9611,1412,6518,1681,35
13,Anderson,10098,63,0,NaN,8,9,46,721,147,454,120,5


In [9]:
from sklearn.model_selection import train_test_split
# Instantiate and fit our model.
regr = linear_model.LinearRegression()
Y = df['Property\ncrime'].values.reshape(-1, 1)
X = df[['Violent\ncrime','Robbery','Burglary']]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)
regr.fit(X_train, Y_train)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X_test, Y_test))
# ompute adjusted-r-squared
print(1 - (1-regr.score(X_test, Y_test))*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1))


Coefficients: 
 [[-0.36467701  5.9839037   3.1030044 ]]

Intercept: 
 [ 140.07293992]

R-squared:
0.666737578677
0.659331747092


In [38]:
crime['mukduk'] = crime['Murder and\nnonnegligent\nmanslaughter'].apply(mukduk)

crime['Rob'] = crime['Robbery'].apply(robbery)
    
crime['propertycrime'] = crime.Population + crime.Population*crime.Population + crime.mukduk + crime.Rob

crime.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 16 columns):
City                                    0 non-null object
Population                              0 non-null float64
Violent
crime                           0 non-null object
Murder and
nonnegligent
manslaughter    0 non-null object
Rape
(revised
definition)1              0 non-null object
Rape
(legacy
definition)2               0 non-null object
Robbery                                 0 non-null object
Aggravated
assault                      0 non-null object
Property
crime                          0 non-null object
Burglary                                0 non-null object
Larceny-
theft                          0 non-null object
Motor
vehicle
theft                     0 non-null object
Arson3                                  0 non-null object
mukduk                                  0 non-null object
Rob                                     0 non-null object
propertycrime                       